# Kraftlinjer og transformatorer som blir benyttet i kart
Denne filen lager nettdata som benyttes i kartvisningen i Power BI.

Data er hentet fra NVE atlas.

In [38]:
import geopandas as gpd
from shapely.geometry import Polygon
import pandas as pd

In [39]:
gdf_linje = gpd.read_file("assets/input/NVE_Kraftlinje.geojson")
gdf_trafo = gpd.read_file("assets/input/NVE_Transformatorstasjon.geojson")

# Sett dataene sammen
gdf = gpd.GeoDataFrame(pd.concat([gdf_linje, gdf_trafo], ignore_index=True))



In [40]:
# Endre farger på kraftlinjene
gdf['stroke'] = gdf.apply(
    lambda x: '#4B6455' if x['nveNettnivaa'] == 3 and x['eier'] in ['BKK AS', 'BKK NETT AS']
    else '#B2002C' if x['eier'] == 'STATNETT SF'
    else '#002762' if x['eier'] in ['BKK AS', 'BKK NETT AS']
    else 'grey',
    axis=1
)

# Endre farger på trafoene
gdf['marker-color'] = gdf['eier'].apply(
    lambda x: '#B2002C' if x == 'STATNETT SF' 
    else '#002762' if x in ['BKK AS', 'BKK NETT AS'] 
    else 'grey'
)




In [41]:
# Definer koordinatene til området som jeg ønsker å visualisere i kartet
coordinates = [
    [4.231698912024626, 61.9088356868493],
    [4.231698912024626, 59.592272496126355],
    [7.181154945960856, 59.592272496126355],
    [7.181154945960856, 61.9088356868493],
    [4.231698912024626, 61.9088356868493]
]

# Lag polygonet fra koordinatene
polygon = Polygon(coordinates)

# Filtrer gdf for å beholde bare de som er innenfor polygonet
gdf = gdf[gdf.geometry.within(polygon)]

In [42]:
# Inkludér kun spesifikke Statnett-transformatorer
statnett_trafo = ['Hove', 'Refsdal', 'Steinsland', 'Modalen', 'Haugsvær', 'Lindås', 'Kollsnes',
                  'Lille Sotra', 'Fana', 'Arna', 'Dale', 'Evanger', 'Samnanger', 'Sima',
                  'Mauranger', 'Blåfalli', 'Oksla', 'Åsen', 'Røldal', 'Nesflaten', 'Sauda',
                  'Børtveit', 'Midtfjellet', 'Stord', 'Sogndal', 'Leirdøla', 'Høyanger', 'Moskog']


gdf = gdf[~(
    (gdf['objektType'] == 'EL_Transformatorstasjon') &
    (gdf['eier'] == 'STATNETT SF') &
    (~gdf['navn'].isin(statnett_trafo))
)]


gdf = gdf[(gdf['eier'] != 'SUNNFJORD ENERGI AS') &     # Selskapet eksisterer ikke lenger
          (gdf['navn'] != '132 HYDRO HØYANGER-MOSKOG') & # Revet for 5-6 år siden
          (gdf['navn'] != 'M 132 MOSKOG - HØYANGER (MIDLERTIDIG)') &
          ~((gdf['nveNettnivaa'] == 1) & (gdf['eier'] != 'STATNETT SF'))]  # Fjerner transmisjonenett som ikke statnett eier


In [43]:
# Behold kun transmisjons- og regionalnett
gdf_regional_transmisjon = gdf[gdf["nveNettnivaa"] != 3]


# Behold kun transmisjonsnett
gdf_statnett = gdf[gdf["eier"] == "STATNETT SF"]

In [44]:
gdf_regional_transmisjon_linjer = gdf_regional_transmisjon[~(
    (gdf_regional_transmisjon['objektType'] == 'EL_Transformatorstasjon') &
    (gdf_regional_transmisjon['nveNettnivaa'] != 1)
)]

# Regional- og transmisjonsnett
gdf_regional_transmisjon_linjer.to_file("assets/output/linjer_trafo_reg_transm_linje.geojson", driver="GeoJSON")

In [45]:
# Lagre den filtrerte GeoDataFrame til en ny GeoJSON-fil
#gdf.to_file("assets/output/linjer_trafo_alle_nettnivaa2.geojson", driver="GeoJSON")
#gdf_regional_transmisjon.to_file("assets/output/linjer_trafo_regional_transmisjon2.geojson", driver="GeoJSON")

# Transmisjonsnett
gdf_statnett.to_file("assets/output/linjer_trafo_transmisjon.geojson", driver="GeoJSON")